In [ ]:
import pandas as pd
import networkx as nx
import random
from ema_workbench.util import ema_logging
from ema_workbench import Samplers, MultiprocessingEvaluator, Policy
from problem_formulation import get_model_for_problem_formulation

def main_sobol():
    # Initialize random seed and logging
    random.seed(1234)
    ema_logging.log_to_stderr(ema_logging.INFO)

    # Configure model and scenario settings
    result = get_model_for_problem_formulation(6)
    if result is None:
        raise ValueError("The problem formulation returned None. Please check the problem_formulation_id.")
    
    model, _ = result
    num_scenarios = 100  # Adjust as needed

    # Define the policy for dike heightening and no RfR
    dike_heightening_policy = {lever.name: 10 if "raise" in lever.name else 0 for lever in model.levers}
    policies = [Policy("dike heightening", **dike_heightening_policy)]

    # Perform Sobol sampling
    with MultiprocessingEvaluator(model, n_processes=-1) as evaluator:
        experiment_data, outcome_data = evaluator.perform_experiments(num_scenarios, policies,
                                                                      uncertainty_sampling=Samplers.SOBOL)

    # Save results to CSV files
    experiment_data.to_csv('data/Output/open_exploration_sobol_experiments.csv')
    pd.DataFrame.from_dict(outcome_data).to_csv('data/Output/open_exploration_sobol_outcomes.csv')

if __name__ == "__main__":
    main_sobol()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from ema_workbench import (Model, RealParameter, CategoricalParameter, ScalarOutcome,
                           IntegerParameter, perform_experiments, ema_logging)
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol

# Configure logging
ema_logging.log_to_stderr(ema_logging.INFO)

# Import the DikeNetwork class from the relevant file
from dike_model_function import DikeNetwork

# Function to get the dike model for Sobol analysis
def get_dike_model():
    function = DikeNetwork()
    dike_model = Model("dikesnet", function=function)

    # Specify uncertainties
    Real_uncert = {"Bmax": [30, 350], "pfail": [0, 1]}  # m and [.]
    cat_uncert_loc = {"Brate": (1.0, 1.5, 10)}
    cat_uncert = {f"discount rate {n}": (1.5, 2.5, 3.5, 4.5) for n in function.planning_steps}
    Int_uncert = {"A.0_ID flood wave shape": [0, 132]}

    uncertainties = []

    for uncert_name in cat_uncert.keys():
        categories = cat_uncert[uncert_name]
        uncertainties.append(CategoricalParameter(uncert_name, categories))

    for uncert_name in Int_uncert.keys():
        uncertainties.append(IntegerParameter(uncert_name, Int_uncert[uncert_name][0], Int_uncert[uncert_name][1]))

    for dike in function.dikelist:
        for uncert_name in Real_uncert.keys():
            name = f"{dike}_{uncert_name}"
            lower, upper = Real_uncert[uncert_name]
            uncertainties.append(RealParameter(name, lower, upper))

        for uncert_name in cat_uncert_loc.keys():
            name = f"{dike}_{uncert_name}"
            categories = cat_uncert_loc[uncert_name]
            uncertainties.append(CategoricalParameter(name, categories))

    dike_model.uncertainties = uncertainties

    # Set levers: No RfR, dike heightening
    levers = [IntegerParameter(f"raise {i}", 1, 10) for i in range(1, 6)]
    dike_model.levers = levers

    # Specify outcomes
    outcomes = [
        ScalarOutcome('Total Costs', kind=ScalarOutcome.MINIMIZE),
        ScalarOutcome('Expected Annual Damage', kind=ScalarOutcome.MINIMIZE),
        ScalarOutcome('Dike Investment Costs', kind=ScalarOutcome.MINIMIZE),
        ScalarOutcome('RfR Total Costs', kind=ScalarOutcome.MINIMIZE),
        ScalarOutcome('Expected Evacuation Costs', kind=ScalarOutcome.MINIMIZE),
        ScalarOutcome('Expected Number of Deaths', kind=ScalarOutcome.MINIMIZE)
    ]
    dike_model.outcomes = outcomes

    return dike_model

# Get the model for Sobol analysis
dike_model = get_dike_model()

# Define the SALib problem
problem = get_SALib_problem(dike_model.uncertainties)

# Perform Sobol experiments
n_exp = 1000
experiments_sobol, outcomes_sobol = perform_experiments(dike_model, scenarios=n_exp, uncertainty_sampling='sobol')

# Analyze the results using Sobol
total_costs = outcomes_sobol['Total Costs']
expected_annual_damage = outcomes_sobol['Expected Annual Damage']
dike_investment_costs = outcomes_sobol['Dike Investment Costs']
rfr_total_costs = outcomes_sobol['RfR Total Costs']
expected_evacuation_costs = outcomes_sobol['Expected Evacuation Costs']
expected_deaths = outcomes_sobol['Expected Number of Deaths']

# Perform Sobol analysis
Si_costs = sobol.analyze(problem, total_costs.flatten(), calc_second_order=True, print_to_console=True)
Si_damage = sobol.analyze(problem, expected_annual_damage.flatten(), calc_second_order=True, print_to_console=True)
Si_investment = sobol.analyze(problem, dike_investment_costs.flatten(), calc_second_order=True, print_to_console=True)
Si_rfr = sobol.analyze(problem, rfr_total_costs.flatten(), calc_second_order=True, print_to_console=True)
Si_evacuation = sobol.analyze(problem, expected_evacuation_costs.flatten(), calc_second_order=True, print_to_console=True)
Si_deaths = sobol.analyze(problem, expected_deaths.flatten(), calc_second_order=True, print_to_console=True)

# Visualize the results
def plot_sobol_indices(Si, title):
    Si_filter = {k: Si[k] for k in ['ST', 'ST_conf', 'S1', 'S1_conf']}
    Si_df = pd.DataFrame(Si_filter, index=problem['names'])

    sns.set_style('white')
    fig, ax = plt.subplots(1)

    indices = Si_df[['S1', 'ST']]
    err = Si_df[['S1_conf', 'ST_conf']]

    indices.plot.bar(yerr=err.values.T, ax=ax)
    fig.set_size_inches(8, 6)
    fig.subplots_adjust(bottom=0.3)
    plt.title(title)
    plt.show()

plot_sobol_indices(Si_costs, 'Sobol Sensitivity Analysis for Total Costs')
plot_sobol_indices(Si_damage, 'Sobol Sensitivity Analysis for Expected Annual Damage')
plot_sobol_indices(Si_investment, 'Sobol Sensitivity Analysis for Dike Investment Costs')
plot_sobol_indices(Si_rfr, 'Sobol Sensitivity Analysis for RfR Total Costs')
plot_sobol_indices(Si_evacuation, 'Sobol Sensitivity Analysis for Expected Evacuation Costs')
plot_sobol_indices(Si_deaths, 'Sobol Sensitivity Analysis for Expected Number of Deaths')
